In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.checkpoint import checkpoint
from transformers import get_linear_schedule_with_warmup

import random 
import qadence as qd
import numpy as np 
import time 

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from geopy.distance import great_circle

import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
# from torchsummary import summary
import datetime

import os
import random
import sys

current_dir = os.getcwd()
sys.path.append(os.path.dirname(current_dir))

from code_base.qpa_utils import *

In [2]:
def set_seed(seed=42):
    torch.manual_seed(seed)  # Set PyTorch seed
    np.random.seed(seed)      # Set NumPy seed
    random.seed(seed)         # Set Python random seed

    # Ensure reproducibility for CUDA (if using GPU)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Disable to ensure consistent runs

# Set a fixed seed
set_seed(58) #56 

## Data processing for the Typhoon dataset

Before diving into the exciting Quantum Parameter Adaptation section, we first need to preprocess the data. Since data processing is not the main focus of this project, we will only provide a brief overview. Interested readers can refer to the original paper ([here](https://www.researchgate.net/publication/357911189_AM-ConvGRU_a_spatio-temporal_model_for_typhoon_path_prediction)) for a detailed explanation and the full classical machine learning model.

<img src="https://github.com/CYLphysics/CYLphysics.github.io/blob/master/assets/images/figure/total_data.png?raw=true"/>


In [3]:

# forecast 24-hour lead time 
pre_seq = 4
batch_size = 128
epochs = 256
min_val_loss = 100
model_name = '../results/model_saver/QPA_ck_64_qnn_depth_20.pkl'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
train = pd.read_csv('../data/CMA_train_'+str(pre_seq*6)+'h.csv', header=None)
test = pd.read_csv('../data/CMA_test_'+str(pre_seq*6)+'h.csv', header=None)


CLIPER_feature =  pd.concat((train, test), axis=0)
CLIPER_feature.reset_index(drop=True, inplace=True)


X_wide_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_wide = X_wide_scaler.fit_transform(CLIPER_feature.iloc[:, 6:])
X_wide_train = X_wide[0: train.shape[0], :]

y = y_scaler.fit_transform(CLIPER_feature.loc[:, 3:4])
y_train = y[0: train.shape[0], :]


reanalysis_type = 'z'
# 0 means now 
# 1 means 6-hour ago
# 2 means 12-hour ago
ahead_times = [0,1,2,3]
pressures = [1000, 750, 500, 250]
sequential_reanalysis_list = []
reanalysis_test_dict = {}
X_deep_scaler_dict = {}


for ahead_time in ahead_times:

    reanalysis_list = []
    for pressure in pressures:
        
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)
        train_reanalysis_csv = pd.read_csv('../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_train_31_31.csv', header=None)
        test_reanalysis_csv = pd.read_csv('../data/ERA_Interim/'+folder+'/'+reanalysis_type+str(pressure)+'_test_31_31.csv', header=None)
        
        train_reanalysis = train_reanalysis_csv[train_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = test_reanalysis_csv[test_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)
        
        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_scaler_dict[scaler_name] = MinMaxScaler()
        X_deep = X_deep_scaler_dict[scaler_name] .fit_transform(reanalysis.loc[:, 5:])
        
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 31, 31)
        reanalysis_list.append(X_deep_final)
    
    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_list.append(X_deep_temp)

X_deep_train = np.concatenate(sequential_reanalysis_list, axis=1)

ahead_time: 0 (8406, 1, 4, 31, 31)
ahead_time: 1 (8406, 1, 4, 31, 31)
ahead_time: 2 (8406, 1, 4, 31, 31)
ahead_time: 3 (8406, 1, 4, 31, 31)


## Construction of training set and validation set

The typhoon data from 2000 to 2014 is used for training, while data from 2015 to 2018 is used for testing.

In [5]:
class TrainLoader(Data.Dataset):
    def __init__(self, X_wide_train, X_deep_train, y_train):
        self.X_wide_train = X_wide_train
        self.X_deep_train = X_deep_train
        self.y_train = y_train
        
    def __getitem__(self, index):
        return [self.X_wide_train[index], self.X_deep_train[index]], self.y_train[index]
    
    def __len__(self):
        return len(self.X_wide_train)

In [6]:
full_train_index = [*range(0, len(X_wide_train))]

train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)


train_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size, shuffle=True)


val_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size, shuffle=True)

## Quantum Parameter Adaptation (QPA)

The implementation of Quantum Parameter Adaptation (QPA) is provided in `code_base/qpa_utils.py`. 

<img src="https://github.com/CYLphysics/CYLphysics.github.io/blob/master/assets/images/figure/qeel.png?raw=true"/>


<!-- Interestingly, the core concept of the QPA is that, we compress only the training parameters during training, using QML technique (more detail in the technical report). Thus, during the inference stage, ie.. the daily usage of the trained model, do not require the usage of quantum computer, since the training result of QPA is a pure classical model. QPA is a Quantum-Train-based (QT-based) method, the comparison of the computational scheme for conventional QML and QT is shown below. Furthermore, as one may observe, the QT-based method don't require the data encoding part of the quantum circuit, since the data is inputted directly into the classical model, thus QT (QPA) also eliminate the issue of data encoding, which is very challenging when the input data is large. Lastly, QT (QPA) utlize the power of Hilbert space, assuming the polynomial number of QNN layers are used, training the classical model with $m$ parameters only requires $polylog(m)$ parameters. (more detail in the section 2 of the technical report.) -->
 
The core concept of QPA is that we compress only the training parameters during training using Quantum Machine Learning (QML) techniques (see the technical report for more details). During inference—i.e., the daily usage of the trained model—no quantum computing resources are required, as the output of QPA is a purely classical model.

QPA is a Quantum-Train-based (QT-based) method, and a comparison of the computational schemes between conventional QML and QT is shown below. Notably, QT-based methods do not require a quantum data encoding process, since the data is directly input into the classical model. This eliminates the data encoding challenge, which becomes increasingly difficult as the input data size grows.

Furthermore, QT (QPA) leverages the power of Hilbert space. Assuming a polynomial number of Quantum Neural Network (QNN) layers are used, training a classical model with $ m $ parameters only requires $ polylog(m)$  parameters (see Section 2 of the technical report for further details).

<img src="https://github.com/CYLphysics/CYLphysics.github.io/blob/master/assets/images/figure/qml_and_qt.png?raw=true"/>


<!-- Thus, in the below notebook block, although the original ML model have about 8399540 trainable parameters, one can see in our QPA only require 216398 parameters (under specific hyperparameter setting). This is only 2.57% of the original model. And the qubit usage is 8, as shown below. The calculation of the qubit usage can be found in the section 2 of the technical report. 
 -->

In the notebook block below, while the original machine learning model has 8,399,540 trainable parameters, our QPA approach—under a specific hyperparameter setting—requires only 216,398 parameters. This is merely 2.57% of the original model’s size, demonstrating significant parameter reduction. Additionally, the qubit usage is only 8, as shown below. The detailed calculation of qubit usage can be found in Section 2 of the technical report.

# Training

In [7]:
net = Net() 

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
total_steps = len(train_dataset) * epochs  
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


print("# of trainable parameters in the current model: ",
    sum(p.numel() for p in net.parameters() if p.requires_grad)
)

print(" # of required qubits in the current QPA setting: ", 
      net.fc1.grand_hypernetwork[0].n_qubit
      )


/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/nn/modules/module.py:1145: UserWarning: Complex modules are a new feature under active development whose design may change, and some modules might not work as expected when using complex tensors as parameters or buffers. Please file an issue at https://github.com/pytorch/pytorch/issues/new?template=bug-report.yml if a complex module does not work as expected.
  warnings.warn(


# of trainable parameters in the current model:  216398
 # of required qubits in the current QPA setting:  8


In [8]:
full_train_index = [*range(0, len(X_wide_train))]


total_train_loss_list = [] 
total_val_loss_list = [] 

for epoch in range(epochs):  # loop over the dataset multiple times
    starttime = datetime.datetime.now()
    train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)
    train_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size,)
    val_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size,)
    # training
    total_train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        since_batch = time.time()
        if torch.cuda.is_available():
            net.cuda()
            # X_wide_train_cuda = batch_x[0].float().cuda()
            # X_deep_train_cuda = batch_x[1].float().cuda()
            X_wide_train_cuda = batch_x[0].to(torch.complex128).cuda()
            X_deep_train_cuda = batch_x[1].to(torch.complex128).cuda()
                
            y_train_cuda = batch_y.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_y = net(X_wide_train_cuda, X_deep_train_cuda)
        loss = criterion(pred_y, y_train_cuda)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # print(f"Step [{step+1}/{len(train_dataset)}], batch time: {time.time() - since_batch:.2f}, train_loss: {loss.item()}")

    # validation
    total_val_loss = 0
    for _,(batch_val_x, batch_val_y) in enumerate(val_dataset):
        
        if torch.cuda.is_available():
            X_wide_val_cuda = batch_val_x[0].float().cuda()
            X_deep_val_cuda = batch_val_x[1].float().cuda()
            y_val_cuda = batch_val_y.cuda()
        
        pred_y = net(X_wide_val_cuda, X_deep_val_cuda)
        val_loss = criterion(pred_y, y_val_cuda)
        total_val_loss += val_loss.item()
    
        # print statistics
    if min_val_loss > total_val_loss:
        torch.save(net.state_dict(), model_name)
        min_val_loss = total_val_loss
    endtime = datetime.datetime.now()
    print('epochs [%d/%d], cost:%.2fs train_loss: %.5f val_loss: %.5f' % 
          (epoch + 1, epochs, (endtime-starttime).seconds, total_train_loss, total_val_loss))

    total_train_loss_list.append(total_train_loss)
    total_val_loss_list.append(total_val_loss)

print('Finished Training')



/home2/chenyuv2/QPA_Typhoon_Trajectory-main/code_base/qpa_utils.py:442: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:305.)
  x = x.to(torch.float32).cuda()
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ct

epochs [1/256], cost:41.00s train_loss: 11.93839 val_loss: 0.98808
epochs [2/256], cost:40.00s train_loss: 9.21993 val_loss: 0.92905
epochs [3/256], cost:40.00s train_loss: 7.01862 val_loss: 0.70735
epochs [4/256], cost:40.00s train_loss: 5.87121 val_loss: 0.61194
epochs [5/256], cost:40.00s train_loss: 5.11536 val_loss: 0.59566
epochs [6/256], cost:40.00s train_loss: 4.71794 val_loss: 0.51833
epochs [7/256], cost:40.00s train_loss: 4.20430 val_loss: 0.49784
epochs [8/256], cost:40.00s train_loss: 3.84581 val_loss: 0.48825
epochs [9/256], cost:39.00s train_loss: 4.67258 val_loss: 0.50280
epochs [10/256], cost:40.00s train_loss: 3.69485 val_loss: 0.39126
epochs [11/256], cost:39.00s train_loss: 3.28804 val_loss: 0.39571
epochs [12/256], cost:40.00s train_loss: 3.37599 val_loss: 0.31459
epochs [13/256], cost:39.00s train_loss: 3.45126 val_loss: 0.35209
epochs [14/256], cost:39.00s train_loss: 3.00901 val_loss: 0.37014
epochs [15/256], cost:40.00s train_loss: 2.94327 val_loss: 0.31443
epo

## Testing

In [9]:
# net.load_state_dict(torch.load(model_name))
years = test[5].unique()
test_list = []

for year in years:
    temp = test[test[5]==year]
    temp = temp.reset_index(drop=True)
    test_list.append(temp)
    
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net()
net = net.to(device)
net.load_state_dict(torch.load(model_name))

/tmp/ipykernel_938526/1928465884.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(model_name))


<All keys matched successfully>

In [10]:

tid_list = []
time_list = [] 
pred_lat_list = []
pred_long_list = [] 
true_lat_list = [] 
true_long_list = []


with torch.no_grad():
    for year, _test in zip(years, test_list):

        print(year, '年:')
        # print("TID ", _test.loc[:,1])
        y_test_lat = _test.loc[:,3]
        
        y_test_long = _test.loc[:,4]
        
        X_wide_test = X_wide_scaler.transform(_test.loc[:,6:])

        final_test_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
                X_deep = reanalysis_test_dict[scaler_name][reanalysis_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 31, 31)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_list.append(X_deep_temp)
        X_deep_test = np.concatenate(final_test_list, axis=1)

        if torch.cuda.is_available():
            X_wide_test = Variable(torch.from_numpy(X_wide_test).float().cuda())
            X_deep_test = Variable(torch.from_numpy(X_deep_test).float().cuda())

        
        tid  = _test.loc[:,1]
        time_ = _test.loc[:,2]
        print("len(tid) = ",len(tid))
        pred = net(X_wide_test, X_deep_test)

        pred = y_scaler.inverse_transform(pred.cpu().detach().numpy())

        pred_lat = pred[:,0]
        pred_long = pred[:,1]
        
        print("len(pred_lat) =", len(pred_lat))
        true_lat = y_test_lat
        true_long = y_test_long

        diff_lat = np.abs(pred_lat - true_lat)
        diff_long = np.abs(pred_long - true_long)

        print('avg lat:', sum(diff_lat)/len(diff_lat))
        print('avg long:', sum(diff_long)/len(diff_long))

        sum_error = []
        for i in range(0, len(pred_lat)):
            sum_error.append(great_circle((pred_lat[i], pred_long[i]), (true_lat[i], true_long[i])).kilometers)

        print('avg distance error:', sum(sum_error)/len(sum_error))
        
        tid_list.append(tid)
        time_list.append(time_)
        pred_lat_list.append(pred_lat)
        pred_long_list.append(pred_long)
        true_lat_list.append(true_lat)
        true_long_list.append(true_long)
        

tid_list_ =  [item for sublist in tid_list for item in sublist]
time_list_ =  [item for sublist in time_list for item in sublist]
pred_lat_list_ =  [item for sublist in pred_lat_list for item in sublist]
pred_long_list_ =  [item for sublist in pred_long_list for item in sublist]
true_lat_list_ =  [item for sublist in true_lat_list for item in sublist]
true_long_list_ =  [item for sublist in true_long_list for item in sublist]



2015 年:
len(tid) =  908


/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


len(pred_lat) = 908
avg lat: 0.9387572965958048
avg long: 1.2682190143064231
avg distance error: 177.25748232018623
2016 年:
len(tid) =  489


/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


len(pred_lat) = 489
avg lat: 0.8768379909616795
avg long: 1.028579577736571
avg distance error: 155.14789132982375
2017 年:
len(tid) =  544


/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


len(pred_lat) = 544
avg lat: 0.935209140181541
avg long: 1.1554999239304504
avg distance error: 168.95143279158262
2018 年:
len(tid) =  806
len(pred_lat) = 806
avg lat: 0.9458796911736569
avg long: 1.186380807578417
avg distance error: 170.38627197423966


/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home2/chenyuv2/anaconda3/envs/qttp/lib/python3.9/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [11]:
id_key = pd.read_csv('../data/raw.csv', header=None)
track_data = [] 

for i in range(len(tid_list_)):

    if len(id_key[id_key[0] == str(tid_list_[i])][11].unique()) == 0:

        track_data.append([
            tid_list_[i], 
            id_key[id_key[0] == tid_list_[i]][11].unique()[0],
            time_list_[i],
            true_lat_list_[i],
            true_long_list_[i],
            pred_lat_list_[i],
            pred_long_list_[i]
            ])
        
    else:

        track_data.append([
            tid_list_[i], 
            id_key[id_key[0] == str(tid_list_[i])][11].unique()[0],
            time_list_[i],
            true_lat_list_[i],
            true_long_list_[i],
            pred_lat_list_[i],
            pred_long_list_[i]
            ])
        

/tmp/ipykernel_938526/142811645.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  id_key = pd.read_csv('../data/raw.csv', header=None)


In [12]:
# Save predicted trajectory data in the testing dataset
import csv

file_path = "../results/QPA_track_data/track_data_QPA_ck_64_qnn_depth_20.csv"
# Define the column headers
headers = ["TID", "KEY", "TIME", "LAT", "LONG", "PRED_LAT", "PRED_LONG"] 

# Write to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(headers)
    
    # Write the data rows
    writer.writerows(track_data)

print(f"CSV file has been written to {file_path}")


CSV file has been written to ../results/QPA_track_data/track_data_QPA_ck_64_qnn_depth_20.csv
